In [2]:
from array import *
import pandas as pd
import sqlalchemy as db

%load_ext sql
%load_ext dotenv
%sql postgresql://$POSTGRES_USER:$POSTGRES_PWD@db/registry

In [4]:
dtype={0:'string', 1:'string', 2:'string', 3:'string', 4:'string', 5:'string', 6:'string', 7:'string', 8:'string', 9:'string', 10:'string', 11:'string'}
registry = pd.read_csv("/w/data/registry-20230630.csv", dtype=dtype)

registry.replace({ pd.NaT:None }, inplace=True)
registry.head(1)
# registry.columns
# registry.info()


,name,description,ministry,cluster,projectOwnerName,projectOnwerEmail,primaryTechnicalLeadName,primaryTechnicalLeadEmail,secondaryTechnicalLeadName,secondaryTechnicalLeadEmail,licencePlate
0,A provisioned project that is edited,test descirption new,AEST,CLAB,Test Abc,test@abc.com,Oamar Kanji,oamarkanji@gmail.com,None,None,d9a24b


In [35]:
conn_str = 'postgresql://app_api_704fkrus:bacon@db/registry'
engine = db.create_engine(conn_str)
connection = engine.connect()

meta_data = db.MetaData()
db.MetaData.reflect(meta_data, bind=engine)
project = meta_data.tables['project']

for i, row in registry.iterrows():
  try:
    stmt = db.insert(project).values(
      project_set_id=row['licencePlate'],
      name=row['name'],
      description=row['description'],
      ministry=row['ministry'],
      cluster=row['cluster'],
      project_owner_name=row['projectOwnerName'],
      project_onwer_email=row['projectOnwerEmail'],
      primary_technical_lead_name=row['primaryTechnicalLeadName'],
      primary_technical_lead_email=row['primaryTechnicalLeadEmail'],
      secondary_technical_lead_name=row['secondaryTechnicalLeadName'],
      secondary_technical_lead_email=row['secondaryTechnicalLeadEmail'])
      
    # print(stmt)
    connection.execute(stmt)
  except Exception as e:
      print(e)

connection.commit()